<a href="https://colab.research.google.com/github/Smalzzo/Projet_HuggingFace/blob/main/TestHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background-color:#e0f7fa; padding: 20px; border-radius: 10px;">
<h2>🔐 Bloc : Vérification de la clé API Hugging Face</h2>

<p><strong>🔍 But :</strong> Ce bloc teste si la clé API Hugging Face est bien chargée depuis un fichier `.env`, puis vérifie sa validité en interrogeant l’API.</p>


<h3>🧾 Résumé :</h3>
<p>
Ce bloc de code permet de :
<ul>
  <li>Charger les variables d’environnement (comme une clé API) depuis un fichier sécurisé appelé <code>.env</code>.</li>
  <li>Récupérer la clé API stockée sous le nom <code>API_KEY</code>.</li>
  <li>Vérifier si cette clé est correcte en appelant une fonction de la bibliothèque <code>huggingface_hub</code>.</li>
  <li>Afficher les informations du compte Hugging Face si la clé est valide, sinon afficher une erreur.</li>
</ul>
</p>

<h3>🖼️ Schéma explicatif (ASCII) :</h3>
<pre>
1. Charger le .env     --->    /content/.../.env
2. Lire la clé API     --->    API_KEY = "hf_xxx"
3. Appel API            --->    whoami(API_KEY)
4. Résultat :
   ├── Si OK     → Clé valide + infos utilisateur
   └── Si erreur → Message "Clé API NotOK ou erreur : ..."
</pre>

<h3>✅ Bonnes pratiques :</h3>
<ul>
  <li>✅ Utiliser un fichier <code>.env</code> pour stocker les clés sensibles et ne jamais les écrire en dur dans le code.</li>
  <li>✅ Utiliser <code>load_dotenv(..., override=True)</code> pour garantir que les valeurs du fichier écrasent les anciennes si nécessaire.</li>
  <li>✅ Protéger le bloc avec un <code>try/except</code> permet de capturer les erreurs et éviter que le programme ne plante brutalement.</li>
  <li>✅ Toujours vérifier que la clé fonctionne avant d’utiliser des fonctions qui nécessitent l’authentification.</li>
</ul>
</div>


In [ ]:
#test clé api
from huggingface_hub import whoami
from dotenv import load_dotenv
import os

load_dotenv("/content/drive/MyDrive/Projet_HuggingFace/.env",override=True)

api_key = os.getenv("API_KEY")

try:
    print(api_key)
    user_info = whoami(api_key)
    print("Clé API Ok!")
    print("infos user :", user_info)
except Exception as e:
    print("Clé API NotOK ou erreur :", e)

<div style="background-color:#e0f7fa; padding: 20px; border-radius: 10px;">

<h2>🤖 Bloc : Chargement d’un modèle SegFormer pour la segmentation sémantique</h2>

<p><strong>🔍 But :</strong> Ce bloc importe les bibliothèques nécessaires et télécharge un modèle préentraîné de segmentation sémantique (SegFormer) depuis la plateforme Hugging Face. Il prépare l’environnement pour traiter des images et effectuer des inférences.</p>

<h3>📐 Concepts ou formules associées :</h3>
<p>
Ce bloc s’appuie sur les concepts de réseaux de neurones convolutifs (CNN) pour la segmentation sémantique.  
La sortie du modèle est une carte de segmentation où chaque pixel est associé à une classe. Le principe peut être résumé ainsi :
</p>

<p>
\[
\text{Segmentation}(x) = \arg\max(\text{logits}(x)) \quad \text{(par pixel)}
\]
</p>

<p>
On utilise aussi une interpolation pour mettre à l’échelle les sorties du modèle :
\[
\text{logits\_resized} = \text{F.interpolate}(\text{logits}, \text{target\_size})
\]
</p>

<h3>🧾 Résumé :</h3>
<ul>
  <li>📦 Le modèle utilisé s’appelle <code>mattmdjaga/segformer_b2_clothes</code> et est spécialisé dans la segmentation d’éléments vestimentaires.</li>
  <li>📥 On télécharge le processeur (prétraitement) et le modèle (réseau de neurones) depuis Hugging Face.</li>
  <li>🧠 Ces outils vont permettre, plus tard, de traiter une image et d’obtenir une segmentation pixel par pixel.</li>
  <li>📚 Plusieurs bibliothèques comme <code>torch.nn.functional</code>, <code>numpy</code> ou <code>PIL</code> sont importées pour les traitements d’image.</li>
</ul>

<h3>🖼️ Schéma explicatif (ASCII simplifié) :</h3>
<pre>
       Image input
           ↓
  [SegformerImageProcessor]
           ↓
       [Model: SegFormer]
           ↓
  → Carte de segmentation
     (1 classe par pixel)
</pre>

<h3>✅ Bonnes pratiques :</h3>
<ul>
  <li>✅ Toujours utiliser la méthode <code>from_pretrained()</code> pour récupérer les modèles hébergés sur Hugging Face en toute sécurité.</li>
  <li>✅ Vérifier la compatibilité entre le processeur (prétraitement) et le modèle (réseau) en utilisant le même nom de modèle.</li>
  <li>✅ Charger les bibliothèques utiles dès le départ pour éviter des erreurs lors de l’exécution du pipeline complet.</li>
</ul>

</div>


In [ ]:
# utilisation du Model Hugging Face

from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd


# on charge le model
# le request à l'api hugginFace est encaspulé dans SegformerImageProcessor et AutoModelForSemanticSegmentation
# il suffit ensuite de lui donner le model a utiliser "mattmdjaga/segformer_b2_clothes"
processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")


<div style="background-color:#f5faff; padding: 20px; border-radius: 10px;">

<h2>🧩 Bloc : Chargement d’images, prédiction avec un modèle de segmentation, et comparaison des prédictions</h2>

<p><strong>🔍 But :</strong> Ce bloc lit des images et leurs masques dans des répertoires, applique un modèle SegFormer pour obtenir une segmentation prédite, affiche les résultats visuellement, puis calcule et compare les distributions des classes (labels) présentes dans le masque réel et la prédiction.</p>


<h3>🧾 Résumé clair :</h3>
<ul>
  <li>🔹 Le code change de répertoire vers le dossier contenant les données.</li>
  <li>🔹 Il récupère les chemins vers les images et leurs masques.</li>
  <li>🔹 Pour chaque image, il trouve le masque correspondant et les charge.</li>
  <li>🔹 L’image est passée dans un modèle SegFormer qui prédit une segmentation (carte de classes).</li>
  <li>🔹 On affiche côte à côte le masque réel et la prédiction du modèle.</li>
  <li>🔹 On calcule combien de pixels appartiennent à chaque classe (label) dans le masque réel et dans la prédiction.</li>
  <li>🔹 On retire les pixels de fond (label 0) pour avoir des pourcentages pertinents.</li>
  <li>🔹 On affiche un histogramme comparatif pour voir où le modèle est cohérent ou non.</li>
</ul>

<h3>🖼️ Schéma explicatif simplifié (ASCII) :</h3>
<pre>
Répertoire "IMG"                 Répertoire "MASK"
       │                                │
       ▼                                ▼
 +-------------+                +----------------+
 | image_0.png |   ======⇒      | mask_0.png     |
 +-------------+                +----------------+
       │                                │
       ▼                                ▼
    Chargement                   Chargement (niveau de gris)
       │                                │
       └──────►  Inférence avec le modèle  ◄──────┘
                          │
                          ▼
             +--------------------------+
             |  prédiction (segmentation) |
             +--------------------------+
                          │
                          ▼
     Comparaison et histogramme (% pixels par classe)
</pre>

<h3>✅ Bonnes pratiques :</h3>
<ul>
  <li>✅ Vérifier le répertoire courant avec <code>os.getcwd()</code> et le modifier avec <code>os.chdir()</code> si nécessaire.</li>
  <li>✅ Filtrer les pixels de fond (label 0) pour éviter des biais dans l’analyse.</li>
  <li>✅ Visualiser les résultats (masque vs. prédiction) permet une première validation rapide.</li>
  <li>✅ Comparer les pourcentages par classe dans un DataFrame rend l’analyse plus lisible et exploitable.</li>
  <li>✅ Bien gérer les noms des fichiers avec <code>os.path.splitext</code> et <code>replace()</code> pour relier image et masque.</li>
</ul>

</div>


In [ ]:

print("Répertoire courant :", os.getcwd())

os.chdir("/content/drive/MyDrive/Projet_HuggingFace")

#chemins des fichiers images et des masks
images_dir = os.path.join("DataImg", "IMG")
masks_dir  = os.path.join("DataImg", "MASK")

results = []

# on parcour le repertoire IMG
#pour chaque image trouvé on récupère son indice
# et on va chercher dans le repertour MASK les mask avec le meme indice
for fname in os.listdir(images_dir)[:3]:

    img_path = os.path.join(images_dir, fname)
    base     = os.path.splitext(fname)[0]
    # ca permet de récuperer le mask qui correspond à son image : image_0 =mask_0
    mask_name = f"mask{base.replace('image','')}.png"
    mask_path = os.path.join(masks_dir, mask_name)

    #si il maque un mask on passe  à l image suivante
    if not os.path.exists(mask_path):
        continue

    # Chargement en mettant les pixel en RGB pour les image et en gris( mono couleur pour avoir un anitier pour chaque pixel) pour les mask
    #segform utilise le RGB
    image = Image.open(img_path).convert("RGB")
    mask  = Image.open(mask_path).convert("L")





    #inference, on injecte l'image dans le model
    #on recupere la segmentation predite
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    upsampled_logits = F.interpolate(logits, size=image.size[::-1], mode="bilinear", align_corners=False)
    pred_seg = upsampled_logits.argmax(dim=1)[0]


    # visualisation du mask et de la segmentation  predite
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(mask)
    plt.title("Image mask")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(pred_seg)
    plt.title("Segmentation")
    plt.axis("off")
    plt.suptitle(img_path)
    plt.show()


    #tableau contenat les pixels par type d' Id
    mask_arr = np.array(mask)
    pred_arr = np.array(pred_seg)

    #Calcul des distributions
    labels     = sorted(set(np.unique(mask_arr)) | set(np.unique(pred_seg)))
    mask_counts = [(mask_arr == l).sum() for l in labels]
    pred_counts = [(pred_arr == l).sum() for l in labels]
    total       = mask_arr.size



    dframe = pd.DataFrame({
        'label_id':     labels,
        'mask_count':   mask_counts,
        'mask_pct(%)':  np.array(mask_counts)/total*100,
        'pred_count':   pred_counts,
        'pred_pct(%)':  np.array(pred_counts)/total*100,
    })


    # je retire les  pixel de fond id=0
    dframe_no_bg = dframe[dframe['label_id'] != 0].reset_index(drop=True).copy()

    total_no_bg = dframe_no_bg['mask_count'].sum()
    dframe_no_bg['mask_pct(%)'] = dframe_no_bg['mask_count']  / total_no_bg * 100
    dframe_no_bg['pred_pct(%)'] = dframe_no_bg['pred_count']  / total_no_bg * 100

    display(dframe_no_bg)

    df = dframe_no_bg

    x = df['label_id'].astype(str)
    plt.figure(figsize=(10,5))
    w = 0.35
    plt.bar(np.arange(len(x)) - w/2, df['mask_pct(%)'], width=w, label='Mask')
    plt.bar(np.arange(len(x)) + w/2, df['pred_pct(%)'], width=w, label='Pred')
    plt.xticks(np.arange(len(x)), x, rotation=90)
    plt.ylabel("Pourcentage de pixels (%)")
    plt.xlabel("Label ID")
    plt.legend()
    plt.tight_layout()
    plt.show()





<div style="background-color:#f0f8ff; padding:20px; border-radius:10px;">

<h2>📊 Bloc : Fonctions d'évaluation pour la segmentation d'image</h2>

<p><strong>🔍 But :</strong> Ce bloc définit trois fonctions utiles pour évaluer les performances d’un modèle de segmentation :
<ul>
  <li><code>accuracy</code> : précision des prédictions hors fond</li>
  <li><code>iou_class</code> : Intersection over Union (IoU) par classe</li>
  <li><code>nuagePixel</code> : visualisation des pixels réels vs prédits (hors fond)</li>
</ul>
</p>

<h3>📐 Formules mathématiques associées :</h3>

<p><strong>1. Précision (hors fond)</strong></p>
<p>
$$
\text{Accuracy} = \frac{\text{nb pixels corrects (hors fond)}}{\text{nb total pixels (hors fond)}}
$$
</p>

<p><strong>2. Intersection over Union (IoU) :</strong></p>
<p>
$$
\text{IoU}_{c} = \frac{|P_c \cap M_c|}{|P_c \cup M_c|}
$$
où $$
\text {P}_{c} = \text{pixels prédits de classe c,} \text{M}_{c}= \text{pixels du masque de classe c}
$$
</p>

<h3>🧾 Résumé clair :</h3>
<ul>
  <li>🧮 <code>accuracy</code> mesure le pourcentage de pixels bien classés, sans compter ceux du fond (classe 0).</li>
  <li>📐 <code>iou_class</code> compare, pour chaque classe, combien de pixels sont à la fois dans la prédiction et le masque (intersection), par rapport à ceux présents dans au moins un des deux (union).</li>
  <li>📊 <code>nuagePixel</code> affiche visuellement les pixels du masque et de la prédiction, sous forme de points colorés selon la classe (sans le fond).</li>
</ul>



<h3>✅ Bonnes pratiques :</h3>
<ul>
  <li>✅ Toujours exclure la classe qui pourrait biaiser la métrique pour évaluer la performance réelle sur les objets d’intérêt. ici c est le background ID=0</li>
  <li>✅ Utiliser l’IoU est plus informatif que l’accuracy pour des tâches de segmentation avec des objets partiellement présents.</li>
  <li>✅ Le nuage de pixels est utile pour déceler visuellement les décalages ou les erreurs globales de segmentation.</li>
  <li>✅ La fonction <code>meshgrid</code> permet de récupérer facilement les coordonnées x, y de chaque pixel pour les scatter plots.</li>
</ul>

</div>


In [ ]:
# Fonctions métriques

def accuracy(pred, mk, px_id=0):
    mask = (mk != px_id)
    # Nombre de pixels qui ne sont pas du fond (id!=0)
    total = mask.sum()
    correct = ((pred == mk) & mask).sum()
    return correct / total

def iou_class(pred, mk, n_classes, ignore_index=0):
    ious = []
    for c in range(n_classes):
        if c == ignore_index:
            continue
        p = (pred == c)&(pred!=ignore_index)
        m = (mk   == c)&(mk!=ignore_index)
        inter = np.logical_and(p, m).sum()
        union = np.logical_or(p, m).sum()
        if union > 0:
            ious.append(inter / union)
    return np.array(ious)


def nuagePixel(pred_arr,mask_arr):
  h, w = mask_arr.shape
  x, y = np.meshgrid(np.arange(w), np.arange(h))
  x = x.flatten()
  y = y.flatten()


  labels_mask = mask_arr.flatten()
  labels_pred = pred_arr.flatten()

  # Exclure la classe de fond (0)
  mask_indices = labels_mask != 0
  pred_indices = labels_pred != 0

  # Tracé des nuages de points
  fig, axs = plt.subplots(1, 2, figsize=(16, 7))

  axs[0].scatter(x[mask_indices], y[mask_indices], c=labels_mask[mask_indices], cmap='tab20', s=1)
  axs[0].invert_yaxis()
  axs[0].set_title("Nuage de pixels - Masque réel (sans fond)")
  axs[0].set_xlabel("X")
  axs[0].set_ylabel("Y")

  axs[1].scatter(x[pred_indices], y[pred_indices], c=labels_pred[pred_indices], cmap='tab20', s=1)
  axs[1].invert_yaxis()
  axs[1].set_title("Nuage de pixels - Prédiction (sans fond)")
  axs[1].set_xlabel("X")
  axs[1].set_ylabel("Y")

  plt.tight_layout()
  plt.show()




<div style="background-color:#f0f8ff; padding: 20px; border-radius: 10px;">

<h2>📊 Bloc : Évaluation automatique des métriques de segmentation (PA et mIoU)</h2>

<h3>🔍 But du bloc</h3>
<p>
Ce bloc évalue automatiquement les performances d’un modèle de segmentation d’image. Pour chaque image d’un répertoire, il :
</p>
<ul>
  <li>🖼️ Charge l’image et son masque de vérité terrain (masque réel)</li>
  <li>🤖 Applique le modèle de segmentation (inférence)</li>
  <li>📐 Calcule les métriques <strong>Pixel Accuracy</strong> (PA) et <strong>mean IoU</strong> (mIoU)</li>
  <li>📈 Affiche une courbe de performance image par image</li>
  <li>📊 Calcule et affiche les métriques moyennes sur tout le jeu de test</li>
</ul>

<h3>📐 Formules mathématiques associées</h3>

<p><strong>Pixel Accuracy (PA)</strong> :</p>
<p>
$$
\text{PA} = \frac{\text{nb pixels bien prédits (hors fond)}}{\text{nb total pixels (hors fond)}}
$$
</p>

<p><strong>Intersection over Union (IoU)</strong> pour une classe <i>c</i> :</p>
<p>
$$
\text{IoU}_c = \frac{|P_c \cap M_c|}{|P_c \cup M_c|}
$$
</p>

<p><strong>Mean IoU (mIoU)</strong> :</p>
<p>
$$
\text{mIoU} = \frac{1}{N} \sum_{c=1}^{N} \text{IoU}_c
$$
où \( C \) est le nombre de classes (hors fond).
</p>

<h3>🧾 Résumé pour débutants</h3>
<ul>
  <li>🔁 Le code lit toutes les images d’un dossier et récupère leur masque associé.</li>
  <li>🧠 Chaque image est passée dans un modèle de segmentation (par exemple SegFormer).</li>
  <li>🧮 Les métriques PA et mIoU sont calculées pour chaque image.</li>
  <li>📊 Les résultats sont sauvegardés dans deux listes (<code>Metric_PA</code> et <code>Metric_IOU</code>).</li>
  <li>📉 À la fin, une courbe compare les performances image par image.</li>
  <li>📢 Enfin, une moyenne globale des métriques est affichée pour tout le dataset.</li>
</ul>

<h3>🖼️ Schéma explicatif (pseudo-code ASCII)</h3>

<pre>
Pour chaque image du répertoire :
    ├── Charger l'image et le masque
    ├── Appliquer le modèle (inférence)
    ├── Calculer :
    │     ├── Pixel Accuracy (PA)
    │     └── Mean IoU (mIoU)
    └── Ajouter PA et mIoU dans des listes

À la fin :
    ├── Affichage de la courbe PA/mIoU par image
    └── Moyenne des scores sur toutes les images
</pre>

<h3>✅ Bonnes pratiques</h3>
<ul>
  <li>✅ Toujours vérifier l’existence du masque avant de traiter une image.</li>
  <li>✅ Exclure la classe de fond (ID = 0) pour ne pas fausser les métriques.</li>
  <li>✅ Sauvegarder les métriques dans des structures simples (listes) pour analyse ultérieure.</li>
  <li>✅ Visualiser les performances permet de repérer les images mal traitées.</li>
  <li>✅ Utiliser <code>np.nanmean()</code> pour éviter les erreurs sur des classes absentes.</li>
</ul>

</div>


In [ ]:
#metrique
Metric_PA  = []
Metric_IOU = []


# on parcour le repertoire IMG
#pour chaque image trouvé on récupère son indice
# et on va chercher dans le repertour MASK les mask avec le meme indice
for fname in os.listdir(images_dir):

    img_path = os.path.join(images_dir, fname)
    base     = os.path.splitext(fname)[0]
    # ca permet de récuperer le mask qui correspond à son image : image_0 =mask_0
    mask_name = f"mask{base.replace('image','')}.png"
    mask_path = os.path.join(masks_dir, mask_name)

    #si il maque un mask on passe  à l image suivante
    if not os.path.exists(mask_path):
        continue

    # Chargement en mettant les pixel en RGB pour les image et en gris( mono couleur pour avoir un anitier pour chaque pixel) pour les mask
    #segform utilise le RGB
    image = Image.open(img_path).convert("RGB")
    mask  = Image.open(mask_path).convert("L")

    #inference, on injecte l'image dans le model
    #on recupere la segmentation predite
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    upsampled_logits = F.interpolate(logits, size=image.size[::-1], mode="bilinear", align_corners=False)
    pred_seg = upsampled_logits.argmax(dim=1)[0]


     #tableau contenat les pixels par type d' Id
    mask_arr = np.array(mask)
    pred_arr = np.array(pred_seg)


  # Calcul des métriques
    pam    = accuracy(pred_arr, mask_arr)
    ious  = iou_class(pred_arr, mask_arr, n_classes=len(model.config.id2label), ignore_index=0)
    miou  = np.nanmean(ious)
    #print(f"{fname} → PA={pam:.3f}, mIoU={miou:.3f}")

    Metric_PA.append(pam)
    Metric_IOU.append(miou)



# 5. Metrique global

indices = list(range(len(Metric_PA)))
labels = [f"img_{i}" for i in indices]  # ou tes vrais noms d’image

plt.figure(figsize=(14, 6))
plt.plot(indices, Metric_PA, marker='o', label='Pixel Accuracy (PA)')
plt.plot(indices, Metric_IOU, marker='s', label='Mean IoU (mIoU)', linestyle='--')

plt.xticks(indices, labels, rotation=45)
plt.xlabel("Images")
plt.ylabel("Score")
plt.title("Évolution des métriques par image")
plt.grid(True, linestyle=":")
plt.legend()
plt.tight_layout()
plt.show()


print("=== Metrique global ===")
print(f"Images évaluées : {len(Metric_PA)}")
print(f"Pixel Accuracy (moyenne) : {np.mean(Metric_PA):.4f}")
print(f"Mean IoU (moyenne)       : {np.mean(Metric_IOU):.4f}")


In [ ]:
#ajoute un drive google
from google.colab import drive
drive.mount('/content/drive')

<div style="background-color:#e0f7fa; padding: 20px; border-radius: 10px;">
<h2>🔐 Bloc : Vérification de la clé API Hugging Face</h2>

<p><strong>🔍 But :</strong> Ce bloc teste si la clé API Hugging Face est bien chargée depuis un fichier `.env`, puis vérifie sa validité en interrogeant l’API.</p>


<h3>🧾 Résumé :</h3>
<p>
Ce bloc de code permet de :
<ul>
  <li>Charger les variables d’environnement (comme une clé API) depuis un fichier sécurisé appelé <code>.env</code>.</li>
  <li>Récupérer la clé API stockée sous le nom <code>API_KEY</code>.</li>
  <li>Vérifier si cette clé est correcte en appelant une fonction de la bibliothèque <code>huggingface_hub</code>.</li>
  <li>Afficher les informations du compte Hugging Face si la clé est valide, sinon afficher une erreur.</li>
</ul>
</p>

<h3>🖼️ Schéma explicatif (ASCII) :</h3>
<pre>
1. Charger le .env     --->    /content/.../.env
2. Lire la clé API     --->    API_KEY = "hf_xxx"
3. Appel API            --->    whoami(API_KEY)
4. Résultat :
   ├── Si OK     → Clé valide + infos utilisateur
   └── Si erreur → Message "Clé API NotOK ou erreur : ..."
</pre>

<h3>✅ Bonnes pratiques :</h3>
<ul>
  <li>✅ Utiliser un fichier <code>.env</code> pour stocker les clés sensibles et ne jamais les écrire en dur dans le code.</li>
  <li>✅ Utiliser <code>load_dotenv(..., override=True)</code> pour garantir que les valeurs du fichier écrasent les anciennes si nécessaire.</li>
  <li>✅ Protéger le bloc avec un <code>try/except</code> permet de capturer les erreurs et éviter que le programme ne plante brutalement.</li>
  <li>✅ Toujours vérifier que la clé fonctionne avant d’utiliser des fonctions qui nécessitent l’authentification.</li>
</ul>
</div>


In [ ]:
#test clé api
from huggingface_hub import whoami
from dotenv import load_dotenv
import os

load_dotenv("/content/drive/MyDrive/Projet_HuggingFace/.env",override=True)

api_key = os.getenv("API_KEY")

try:
    print(api_key)
    user_info = whoami(api_key)
    print("Clé API Ok!")
    print("infos user :", user_info)
except Exception as e:
    print("Clé API NotOK ou erreur :", e)

<div style="background-color:#e0f7fa; padding: 20px; border-radius: 10px;">

<h2>🤖 Bloc : Chargement d’un modèle SegFormer pour la segmentation sémantique</h2>

<p><strong>🔍 But :</strong> Ce bloc importe les bibliothèques nécessaires et télécharge un modèle préentraîné de segmentation sémantique (SegFormer) depuis la plateforme Hugging Face. Il prépare l’environnement pour traiter des images et effectuer des inférences.</p>

<h3>📐 Concepts ou formules associées :</h3>
<p>
Ce bloc s’appuie sur les concepts de réseaux de neurones convolutifs (CNN) pour la segmentation sémantique.  
La sortie du modèle est une carte de segmentation où chaque pixel est associé à une classe. Le principe peut être résumé ainsi :
</p>

<p>
\[
\text{Segmentation}(x) = \arg\max(\text{logits}(x)) \quad \text{(par pixel)}
\]
</p>

<p>
On utilise aussi une interpolation pour mettre à l’échelle les sorties du modèle :
\[
\text{logits\_resized} = \text{F.interpolate}(\text{logits}, \text{target\_size})
\]
</p>

<h3>🧾 Résumé :</h3>
<ul>
  <li>📦 Le modèle utilisé s’appelle <code>mattmdjaga/segformer_b2_clothes</code> et est spécialisé dans la segmentation d’éléments vestimentaires.</li>
  <li>📥 On télécharge le processeur (prétraitement) et le modèle (réseau de neurones) depuis Hugging Face.</li>
  <li>🧠 Ces outils vont permettre, plus tard, de traiter une image et d’obtenir une segmentation pixel par pixel.</li>
  <li>📚 Plusieurs bibliothèques comme <code>torch.nn.functional</code>, <code>numpy</code> ou <code>PIL</code> sont importées pour les traitements d’image.</li>
</ul>

<h3>🖼️ Schéma explicatif (ASCII simplifié) :</h3>
<pre>
       Image input
           ↓
  [SegformerImageProcessor]
           ↓
       [Model: SegFormer]
           ↓
  → Carte de segmentation
     (1 classe par pixel)
</pre>

<h3>✅ Bonnes pratiques :</h3>
<ul>
  <li>✅ Toujours utiliser la méthode <code>from_pretrained()</code> pour récupérer les modèles hébergés sur Hugging Face en toute sécurité.</li>
  <li>✅ Vérifier la compatibilité entre le processeur (prétraitement) et le modèle (réseau) en utilisant le même nom de modèle.</li>
  <li>✅ Charger les bibliothèques utiles dès le départ pour éviter des erreurs lors de l’exécution du pipeline complet.</li>
</ul>

</div>


In [ ]:
# utilisation du Model Hugging Face

from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd


# on charge le model
# le request à l'api hugginFace est encaspulé dans SegformerImageProcessor et AutoModelForSemanticSegmentation
# il suffit ensuite de lui donner le model a utiliser "mattmdjaga/segformer_b2_clothes"
processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")


<div style="background-color:#f5faff; padding: 20px; border-radius: 10px;">

<h2>🧩 Bloc : Chargement d’images, prédiction avec un modèle de segmentation, et comparaison des prédictions</h2>

<p><strong>🔍 But :</strong> Ce bloc lit des images et leurs masques dans des répertoires, applique un modèle SegFormer pour obtenir une segmentation prédite, affiche les résultats visuellement, puis calcule et compare les distributions des classes (labels) présentes dans le masque réel et la prédiction.</p>


<h3>🧾 Résumé clair :</h3>
<ul>
  <li>🔹 Le code change de répertoire vers le dossier contenant les données.</li>
  <li>🔹 Il récupère les chemins vers les images et leurs masques.</li>
  <li>🔹 Pour chaque image, il trouve le masque correspondant et les charge.</li>
  <li>🔹 L’image est passée dans un modèle SegFormer qui prédit une segmentation (carte de classes).</li>
  <li>🔹 On affiche côte à côte le masque réel et la prédiction du modèle.</li>
  <li>🔹 On calcule combien de pixels appartiennent à chaque classe (label) dans le masque réel et dans la prédiction.</li>
  <li>🔹 On retire les pixels de fond (label 0) pour avoir des pourcentages pertinents.</li>
  <li>🔹 On affiche un histogramme comparatif pour voir où le modèle est cohérent ou non.</li>
</ul>

<h3>🖼️ Schéma explicatif simplifié (ASCII) :</h3>
<pre>
Répertoire "IMG"                 Répertoire "MASK"
       │                                │
       ▼                                ▼
 +-------------+                +----------------+
 | image_0.png |   ======⇒      | mask_0.png     |
 +-------------+                +----------------+
       │                                │
       ▼                                ▼
    Chargement                   Chargement (niveau de gris)
       │                                │
       └──────►  Inférence avec le modèle  ◄──────┘
                          │
                          ▼
             +--------------------------+
             |  prédiction (segmentation) |
             +--------------------------+
                          │
                          ▼
     Comparaison et histogramme (% pixels par classe)
</pre>

<h3>✅ Bonnes pratiques :</h3>
<ul>
  <li>✅ Vérifier le répertoire courant avec <code>os.getcwd()</code> et le modifier avec <code>os.chdir()</code> si nécessaire.</li>
  <li>✅ Filtrer les pixels de fond (label 0) pour éviter des biais dans l’analyse.</li>
  <li>✅ Visualiser les résultats (masque vs. prédiction) permet une première validation rapide.</li>
  <li>✅ Comparer les pourcentages par classe dans un DataFrame rend l’analyse plus lisible et exploitable.</li>
  <li>✅ Bien gérer les noms des fichiers avec <code>os.path.splitext</code> et <code>replace()</code> pour relier image et masque.</li>
</ul>

</div>


In [ ]:

print("Répertoire courant :", os.getcwd())

os.chdir("/content/drive/MyDrive/Projet_HuggingFace")

#chemins des fichiers images et des masks
images_dir = os.path.join("DataImg", "IMG")
masks_dir  = os.path.join("DataImg", "MASK")

results = []

# on parcour le repertoire IMG
#pour chaque image trouvé on récupère son indice
# et on va chercher dans le repertour MASK les mask avec le meme indice
for fname in os.listdir(images_dir)[:3]:

    img_path = os.path.join(images_dir, fname)
    base     = os.path.splitext(fname)[0]
    # ca permet de récuperer le mask qui correspond à son image : image_0 =mask_0
    mask_name = f"mask{base.replace('image','')}.png"
    mask_path = os.path.join(masks_dir, mask_name)

    #si il maque un mask on passe  à l image suivante
    if not os.path.exists(mask_path):
        continue

    # Chargement en mettant les pixel en RGB pour les image et en gris( mono couleur pour avoir un anitier pour chaque pixel) pour les mask
    #segform utilise le RGB
    image = Image.open(img_path).convert("RGB")
    mask  = Image.open(mask_path).convert("L")





    #inference, on injecte l'image dans le model
    #on recupere la segmentation predite
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    upsampled_logits = F.interpolate(logits, size=image.size[::-1], mode="bilinear", align_corners=False)
    pred_seg = upsampled_logits.argmax(dim=1)[0]


    # visualisation du mask et de la segmentation  predite
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(mask)
    plt.title("Image mask")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(pred_seg)
    plt.title("Segmentation")
    plt.axis("off")
    plt.suptitle(img_path)
    plt.show()


    #tableau contenat les pixels par type d' Id
    mask_arr = np.array(mask)
    pred_arr = np.array(pred_seg)

    #Calcul des distributions
    labels     = sorted(set(np.unique(mask_arr)) | set(np.unique(pred_seg)))
    mask_counts = [(mask_arr == l).sum() for l in labels]
    pred_counts = [(pred_arr == l).sum() for l in labels]
    total       = mask_arr.size



    dframe = pd.DataFrame({
        'label_id':     labels,
        'mask_count':   mask_counts,
        'mask_pct(%)':  np.array(mask_counts)/total*100,
        'pred_count':   pred_counts,
        'pred_pct(%)':  np.array(pred_counts)/total*100,
    })


    # je retire les  pixel de fond id=0
    dframe_no_bg = dframe[dframe['label_id'] != 0].reset_index(drop=True).copy()

    total_no_bg = dframe_no_bg['mask_count'].sum()
    dframe_no_bg['mask_pct(%)'] = dframe_no_bg['mask_count']  / total_no_bg * 100
    dframe_no_bg['pred_pct(%)'] = dframe_no_bg['pred_count']  / total_no_bg * 100

    display(dframe_no_bg)

    df = dframe_no_bg

    x = df['label_id'].astype(str)
    plt.figure(figsize=(10,5))
    w = 0.35
    plt.bar(np.arange(len(x)) - w/2, df['mask_pct(%)'], width=w, label='Mask')
    plt.bar(np.arange(len(x)) + w/2, df['pred_pct(%)'], width=w, label='Pred')
    plt.xticks(np.arange(len(x)), x, rotation=90)
    plt.ylabel("Pourcentage de pixels (%)")
    plt.xlabel("Label ID")
    plt.legend()
    plt.tight_layout()
    plt.show()





<div style="background-color:#f0f8ff; padding:20px; border-radius:10px;">

<h2>📊 Bloc : Fonctions d'évaluation pour la segmentation d'image</h2>

<p><strong>🔍 But :</strong> Ce bloc définit trois fonctions utiles pour évaluer les performances d’un modèle de segmentation :
<ul>
  <li><code>accuracy</code> : précision des prédictions hors fond</li>
  <li><code>iou_class</code> : Intersection over Union (IoU) par classe</li>
  <li><code>nuagePixel</code> : visualisation des pixels réels vs prédits (hors fond)</li>
</ul>
</p>

<h3>📐 Formules mathématiques associées :</h3>

<p><strong>1. Précision (hors fond)</strong></p>
<p>
$$
\text{Accuracy} = \frac{\text{nb pixels corrects (hors fond)}}{\text{nb total pixels (hors fond)}}
$$
</p>

<p><strong>2. Intersection over Union (IoU) :</strong></p>
<p>
$$
\text{IoU}_{c} = \frac{|P_c \cap M_c|}{|P_c \cup M_c|}
$$
où $$
\text {P}_{c} = \text{pixels prédits de classe c,} \text{M}_{c}= \text{pixels du masque de classe c}
$$
</p>

<h3>🧾 Résumé clair :</h3>
<ul>
  <li>🧮 <code>accuracy</code> mesure le pourcentage de pixels bien classés, sans compter ceux du fond (classe 0).</li>
  <li>📐 <code>iou_class</code> compare, pour chaque classe, combien de pixels sont à la fois dans la prédiction et le masque (intersection), par rapport à ceux présents dans au moins un des deux (union).</li>
  <li>📊 <code>nuagePixel</code> affiche visuellement les pixels du masque et de la prédiction, sous forme de points colorés selon la classe (sans le fond).</li>
</ul>



<h3>✅ Bonnes pratiques :</h3>
<ul>
  <li>✅ Toujours exclure la classe qui pourrait biaiser la métrique pour évaluer la performance réelle sur les objets d’intérêt. ici c est le background ID=0</li>
  <li>✅ Utiliser l’IoU est plus informatif que l’accuracy pour des tâches de segmentation avec des objets partiellement présents.</li>
  <li>✅ Le nuage de pixels est utile pour déceler visuellement les décalages ou les erreurs globales de segmentation.</li>
  <li>✅ La fonction <code>meshgrid</code> permet de récupérer facilement les coordonnées x, y de chaque pixel pour les scatter plots.</li>
</ul>

</div>


In [ ]:
# Fonctions métriques

def accuracy(pred, mk, px_id=0):
    mask = (mk != px_id)
    # Nombre de pixels qui ne sont pas du fond (id!=0)
    total = mask.sum()
    correct = ((pred == mk) & mask).sum()
    return correct / total

def iou_class(pred, mk, n_classes, ignore_index=0):
    ious = []
    for c in range(n_classes):
        if c == ignore_index:
            continue
        p = (pred == c)&(pred!=ignore_index)
        m = (mk   == c)&(mk!=ignore_index)
        inter = np.logical_and(p, m).sum()
        union = np.logical_or(p, m).sum()
        if union > 0:
            ious.append(inter / union)
    return np.array(ious)


def nuagePixel(pred_arr,mask_arr):
  h, w = mask_arr.shape
  x, y = np.meshgrid(np.arange(w), np.arange(h))
  x = x.flatten()
  y = y.flatten()


  labels_mask = mask_arr.flatten()
  labels_pred = pred_arr.flatten()

  # Exclure la classe de fond (0)
  mask_indices = labels_mask != 0
  pred_indices = labels_pred != 0

  # Tracé des nuages de points
  fig, axs = plt.subplots(1, 2, figsize=(16, 7))

  axs[0].scatter(x[mask_indices], y[mask_indices], c=labels_mask[mask_indices], cmap='tab20', s=1)
  axs[0].invert_yaxis()
  axs[0].set_title("Nuage de pixels - Masque réel (sans fond)")
  axs[0].set_xlabel("X")
  axs[0].set_ylabel("Y")

  axs[1].scatter(x[pred_indices], y[pred_indices], c=labels_pred[pred_indices], cmap='tab20', s=1)
  axs[1].invert_yaxis()
  axs[1].set_title("Nuage de pixels - Prédiction (sans fond)")
  axs[1].set_xlabel("X")
  axs[1].set_ylabel("Y")

  plt.tight_layout()
  plt.show()




<div style="background-color:#f0f8ff; padding: 20px; border-radius: 10px;">

<h2>📊 Bloc : Évaluation automatique des métriques de segmentation (PA et mIoU)</h2>

<h3>🔍 But du bloc</h3>
<p>
Ce bloc évalue automatiquement les performances d’un modèle de segmentation d’image. Pour chaque image d’un répertoire, il :
</p>
<ul>
  <li>🖼️ Charge l’image et son masque de vérité terrain (masque réel)</li>
  <li>🤖 Applique le modèle de segmentation (inférence)</li>
  <li>📐 Calcule les métriques <strong>Pixel Accuracy</strong> (PA) et <strong>mean IoU</strong> (mIoU)</li>
  <li>📈 Affiche une courbe de performance image par image</li>
  <li>📊 Calcule et affiche les métriques moyennes sur tout le jeu de test</li>
</ul>

<h3>📐 Formules mathématiques associées</h3>

<p><strong>Pixel Accuracy (PA)</strong> :</p>
<p>
$$
\text{PA} = \frac{\text{nb pixels bien prédits (hors fond)}}{\text{nb total pixels (hors fond)}}
$$
</p>

<p><strong>Intersection over Union (IoU)</strong> pour une classe <i>c</i> :</p>
<p>
$$
\text{IoU}_c = \frac{|P_c \cap M_c|}{|P_c \cup M_c|}
$$
</p>

<p><strong>Mean IoU (mIoU)</strong> :</p>
<p>
$$
\text{mIoU} = \frac{1}{N} \sum_{c=1}^{N} \text{IoU}_c
$$
où \( C \) est le nombre de classes (hors fond).
</p>

<h3>🧾 Résumé pour débutants</h3>
<ul>
  <li>🔁 Le code lit toutes les images d’un dossier et récupère leur masque associé.</li>
  <li>🧠 Chaque image est passée dans un modèle de segmentation (par exemple SegFormer).</li>
  <li>🧮 Les métriques PA et mIoU sont calculées pour chaque image.</li>
  <li>📊 Les résultats sont sauvegardés dans deux listes (<code>Metric_PA</code> et <code>Metric_IOU</code>).</li>
  <li>📉 À la fin, une courbe compare les performances image par image.</li>
  <li>📢 Enfin, une moyenne globale des métriques est affichée pour tout le dataset.</li>
</ul>

<h3>🖼️ Schéma explicatif (pseudo-code ASCII)</h3>

<pre>
Pour chaque image du répertoire :
    ├── Charger l'image et le masque
    ├── Appliquer le modèle (inférence)
    ├── Calculer :
    │     ├── Pixel Accuracy (PA)
    │     └── Mean IoU (mIoU)
    └── Ajouter PA et mIoU dans des listes

À la fin :
    ├── Affichage de la courbe PA/mIoU par image
    └── Moyenne des scores sur toutes les images
</pre>

<h3>✅ Bonnes pratiques</h3>
<ul>
  <li>✅ Toujours vérifier l’existence du masque avant de traiter une image.</li>
  <li>✅ Exclure la classe de fond (ID = 0) pour ne pas fausser les métriques.</li>
  <li>✅ Sauvegarder les métriques dans des structures simples (listes) pour analyse ultérieure.</li>
  <li>✅ Visualiser les performances permet de repérer les images mal traitées.</li>
  <li>✅ Utiliser <code>np.nanmean()</code> pour éviter les erreurs sur des classes absentes.</li>
</ul>

</div>


In [ ]:
#metrique
Metric_PA  = []
Metric_IOU = []


# on parcour le repertoire IMG
#pour chaque image trouvé on récupère son indice
# et on va chercher dans le repertour MASK les mask avec le meme indice
for fname in os.listdir(images_dir):

    img_path = os.path.join(images_dir, fname)
    base     = os.path.splitext(fname)[0]
    # ca permet de récuperer le mask qui correspond à son image : image_0 =mask_0
    mask_name = f"mask{base.replace('image','')}.png"
    mask_path = os.path.join(masks_dir, mask_name)

    #si il maque un mask on passe  à l image suivante
    if not os.path.exists(mask_path):
        continue

    # Chargement en mettant les pixel en RGB pour les image et en gris( mono couleur pour avoir un anitier pour chaque pixel) pour les mask
    #segform utilise le RGB
    image = Image.open(img_path).convert("RGB")
    mask  = Image.open(mask_path).convert("L")

    #inference, on injecte l'image dans le model
    #on recupere la segmentation predite
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    upsampled_logits = F.interpolate(logits, size=image.size[::-1], mode="bilinear", align_corners=False)
    pred_seg = upsampled_logits.argmax(dim=1)[0]


     #tableau contenat les pixels par type d' Id
    mask_arr = np.array(mask)
    pred_arr = np.array(pred_seg)


  # Calcul des métriques
    pam    = accuracy(pred_arr, mask_arr)
    ious  = iou_class(pred_arr, mask_arr, n_classes=len(model.config.id2label), ignore_index=0)
    miou  = np.nanmean(ious)
    #print(f"{fname} → PA={pam:.3f}, mIoU={miou:.3f}")

    Metric_PA.append(pam)
    Metric_IOU.append(miou)



# 5. Metrique global

indices = list(range(len(Metric_PA)))
labels = [f"img_{i}" for i in indices]  # ou tes vrais noms d’image

plt.figure(figsize=(14, 6))
plt.plot(indices, Metric_PA, marker='o', label='Pixel Accuracy (PA)')
plt.plot(indices, Metric_IOU, marker='s', label='Mean IoU (mIoU)', linestyle='--')

plt.xticks(indices, labels, rotation=45)
plt.xlabel("Images")
plt.ylabel("Score")
plt.title("Évolution des métriques par image")
plt.grid(True, linestyle=":")
plt.legend()
plt.tight_layout()
plt.show()


print("=== Metrique global ===")
print(f"Images évaluées : {len(Metric_PA)}")
print(f"Pixel Accuracy (moyenne) : {np.mean(Metric_PA):.4f}")
print(f"Mean IoU (moyenne)       : {np.mean(Metric_IOU):.4f}")
